In [34]:
import cvxpy as cp
import numpy as np

## Set up constants

# Load data files
accomodations = np.loadtxt("accomodation.csv", delimiter=",",skiprows=1,usecols=(1,2,3,4))

## Set up variables
M = 100000000   # Some arbitrarily large term

# Costs per day (in USD)
avg_cost = accomodations[:,1]

# Travel costs (matrix in USD)
travel_cost = [
    [M, 40, 65, 20, 50, 30, 50, 60],
    [40, M, 105, 40, 130, 70, 165, 130],
    [65, 105, M, 100, 60, 25, 30, 40],
    [20, 40, 100, M, 86, 20, 66, 75],
    [50, 130, 60, 86, M, 100, 80, 20],
    [30, 70, 25, 20, 100, M, 90, 130],
    [50, 165, 30, 66, 80, 90, M, 40],
    [60, 130, 40, 75, 20, 130, 40, M]
]

transport_time = [
    [M,2.50,4.75,2.00,3.00,2.50,2.75,2.00],
    [2.50,M,6.50,3.00,6.00,6.75,6.50,4.75],
    [4.75,6.50,M,6.00,3.00,5.00,1.50,1.75],
    [2.00,3.00,6.00,M,6.00,2.00,6.00,6.00],
    [3.00,6.00,3.00,6.00,M,6.00,2.00,1.00],
    [2.50,6.75,5.00,2.00,6.00,M,5.50,6.00],
    [2.75,6.50,1.50,6.00,2.00,5.50,M,1.00],
    [2.00,4.75,1.75,6.00,1.00,6.00,1.00,M]	
]

is_coastal = accomodations[:,2]
ranking = accomodations[:,3]

# Number of cities
num_cities = len(avg_cost)

# Total budget and length of trip
budget = 1500
trip_length = 14

In [35]:
## Decision variables
x = cp.Variable(num_cities, integer=True) # How long spent in each city

t = cp.Variable((num_cities, num_cities), boolean=True)  # Travel route binary variables -- traveling from city i to j

u = cp.Variable(num_cities,nonneg=True) # artificial variable for eliminating subtours

In [ ]:
## Create objective

# Objective: Minimize total cost
total_cost = (
    sum(avg_cost[i] * x[i]
        for i in range(num_cities)) +
    sum(
        travel_cost[i][j] * t[i, j] 
        for i in range(num_cities) 
        for j in range(num_cities)
    )
)

# Constraints
constraints = [x >= 1, x <= 3] # spend no more than 3 and no less than 1 day in each city

constraints.append(sum(x) == trip_length) # Total days equal to trip duration

constraints.append(sum(
       avg_cost[i] * x[i] +
        sum(travel_cost[i][j] * t[i, j] for j in range(num_cities))
        for i in range(num_cities)
    ) <= budget) # Budget constraint

# Average satisfaction 
constraints.append(sum(ranking[i]*x[i] for i in range(num_cities))/14 >= 8) 

# Spend at least 4 days on the coast
constraints.append(sum(is_coastal[i] * x[i] for i in range(num_cities)) >= 4)

# Travel no more than 5 hours per day
for i in range(num_cities):
    for j in range(num_cities):
      constraints.append(transport_time[i][j] * t[i, j]  <= 5)
 
# Each city visited exactly once
for i in range(num_cities): 
    constraints.append(sum(t[i, j] for j in range(num_cities)) == 1)
    constraints.append(sum(t[j, i] for j in range(num_cities)) == 1)

# No subtours
for i in range(1,num_cities):
   for j in range(1,num_cities):
       constraints.append(u[i] - u[j] + num_cities*t[i,j] <= num_cities-1)

In [37]:
# Solver
problem = cp.Problem(cp.Minimize(total_cost), constraints)
problem.solve(solver=cp.GUROBI)

# Results
print("Optimal Cost:", problem.value)
print("Days in each city:")
print(x.value)
print("Route:")
print(t.value)

Optimal Cost: 605.0
Days in each city:
[1. 3. 1. 2. 2. 1. 1. 3.]
Route:
[[0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]
